In [136]:
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline


In [153]:
def read_data_from_txt(file_name):
    to_read_file = open(file_name, "r")
    paths = to_read_file.read().split('\n')
    path_time = []
    path_x = []
    path_y = []
    for path_data in paths:
        path_data_detail = path_data.split(',')
        if path_data_detail == ['']:
            continue
        t_sec = path_data_detail[0].split(' ')[1].split(':')
        t_sec = float(t_sec[0]) * 60 * 60 +  float(t_sec[1]) * 60 + float(t_sec[2])
        path_time.append(t_sec)
        path_x.append(float(path_data_detail[1]))
        path_y.append(float(path_data_detail[2]))
    print 'length: ', len(path_time)
    return path_time, path_x, path_y

In [154]:
class Path:
    def __init__(self):
        self.t = [] # time
        self.x = [] # position x
        self.y = [] # position y
        self.v = [] # velocity
        self.a = [] # acceleration
        self.d = [] # distance

In [155]:
def calculate_velocity(path):
    v = []
    d = []
    for i in range(len(path.t)-1):
        dt = path.t[i+1] - path.t[i]
        dx = path.x[i+1] - path.x[i]
        dy = path.y[i+1] - path.y[i]
        d_cal = math.sqrt(dx * dx + dy * dy) 
        v_cal = d_cal / dt
        #print dx, dy, v_cal * dt, v_cal, dt
        d.append(d_cal)
        v.append(v_cal)
    return d, v

def calculate_a(path):
    a = []
    for i in range(len(path.t)-2):
        dv = path.v[i+1] - path.v[i]
        dt = path.t[i+2] - path.t[i+1]
        a_cal = dv / dt
        #print dv, dt, a_cal
        a.append(a_cal)
    return a

In [156]:
als_path = Path()
als_path.t, als_path.x, als_path.y = read_data_from_txt("/home/arg/wamv_path/als_path1.txt")
als_path.d, als_path.v = calculate_velocity(als_path)
als_path.a = calculate_a(als_path)

gt_path = Path()
gt_path.t, gt_path.x, gt_path.y = read_data_from_txt("/home/arg/wamv_path/gt_path1.txt")
gt_path.d, gt_path.v = calculate_velocity(gt_path)
gt_path.a = calculate_a(gt_path)

gps_path = Path()
gps_path.t, gps_path.x, gps_path.y = read_data_from_txt("/home/arg/wamv_path/gps_path1.txt")
gps_path.d, gps_path.v = calculate_velocity(gps_path)
gps_path.a = calculate_a(gps_path)

length:  13
length:  381
length:  19


In [163]:
def cal_ape(base_path, target_path):
    error = []
    tp_number = 0
    count = 0
    i = 0
    get_taget_last = False
    while(count < len(base_path.t)):
        bp_dt = base_path.t[i] - base_path.t[0]
        tp_dt = target_path.t[tp_number] - target_path.t[0]
        if bp_dt > tp_dt and (tp_number + 1) < len(target_path.t):
            tp_number += 1
            continue
        if bp_dt <= tp_dt or (tp_number + 1) == len(target_path.t):
            dx = base_path.x[i] - target_path.x[tp_number]
            dy = base_path.y[i] - target_path.y[tp_number]
            e = math.sqrt(dx * dx + dy * dy)
            error.append(e)
            #print i+1, tp_number+1, e
        i += 1
        count += 1
    return error

In [166]:
als_e = cal_ape(gt_path, als_path)
print sum(als_e)
gps_e = cal_ape(gt_path, gps_path)
print sum(gps_e)

94.0529698712
89.3347284283
